# Tasca M10 T01

In [44]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
import re


### - Exercici 1
Realitza web scraping de dues de les tres pàgines web proposades utilitzant BeautifulSoup primer i Selenium després. 

- http://quotes.toscrape.com

- https://www.bolsamadrid.es

- www.wikipedia.es (fes alguna cerca primer i escrapeja algun contingut)

Primero vamos a realizar una request de la página y luego vamos a parsear el contenido con el parseador de html usando BeautifulSoup

In [2]:
url = "https://quotes.toscrape.com/"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")

Con la herramienta de inspección vemos que todas las citas están contenidas en contenedores div que pertenecen a la clase 'quote'. Vamos a seleccionarlas.

In [3]:
quotes=soup.find_all("div", class_="quote")

In [4]:
for quote in quotes:
    print(quote, end="\n"*2)

<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
<span>by <small class="author" itemprop="author">Albert Einstein</small>
<a href="/author/Albert-Einstein">(about)</a>
</span>
<div class="tags">
            Tags:
            <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
<a class="tag" href="/tag/change/page/1/">change</a>
<a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>
<a class="tag" href="/tag/thinking/page/1/">thinking</a>
<a class="tag" href="/tag/world/page/1/">world</a>
</div>
</div>

<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“It is our choices, Harry, that show what we truly are, far more than our abilities.”</span>
<span>by <small class="author" itemprop="author">J.K. 

Ya tenemos las citas recogidas en la variable quotes, pero con mucho html y la información mezclada. Vamos a sacar ahora la información que necesitamos: el texto de la cita, el autor y los tags.

In [12]:
for quote in quotes:
    text=quote.find("span", class_="text").text
    author=quote.find("small", class_="author").text
    tag_html=quote.find_all("a", class_="tag")
    tag_list=[]
    for tag in tag_html:
        tag_list.append(tag.text)
    print(text, author, tag_list)

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.” Albert Einstein ['change', 'deep-thoughts', 'thinking', 'world']
“It is our choices, Harry, that show what we truly are, far more than our abilities.” J.K. Rowling ['abilities', 'choices']
“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.” Albert Einstein ['inspirational', 'life', 'live', 'miracle', 'miracles']
“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.” Jane Austen ['aliteracy', 'books', 'classic', 'humor']
“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.” Marilyn Monroe ['be-yourself', 'inspirational']
“Try not to become a man of success. Rather become a man of value.” Albert Einstein ['adulthood', 'success', 'value']
“It is better to be hated for what you are than to b

Crearemos un dataframe y guardaremos los datos en él

In [14]:
df = pd.DataFrame(columns=['text', 'author', 'tags'])

In [16]:
for quote in quotes:
    text=quote.find("span", class_="text").text
    author=quote.find("small", class_="author").text
    tag_html=quote.find_all("a", class_="tag")
    tag_list=[]
    for tag in tag_html:
        tag_list.append(tag.text)
    df = pd.concat([df, pd.DataFrame([{'text': text, 'author':author, 'tags': tag_list}])], ignore_index=True)

In [17]:
df.head()

,text,author,tags
0,“The world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]"
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"[abilities, choices]"
2,“There are only two ways to live your life. On...,Albert Einstein,"[inspirational, life, live, miracle, miracles]"
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"[aliteracy, books, classic, humor]"
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"[be-yourself, inspirational]"


Como la web tiene 10 páginas y queremos almacenarlas todas, crearemos el mismo proceso que hemos hecho pero dentro de un bucle

In [20]:
df = pd.DataFrame(columns=['text', 'author', 'tags'])

In [21]:
for i in range(1,11):
    url="https://quotes.toscrape.com/page/" + str(i) + "/"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    quotes=soup.find_all("div", class_="quote")
    for quote in quotes:
        text=quote.find("span", class_="text").text
        author=quote.find("small", class_="author").text
        tag_html=quote.find_all("a", class_="tag")
        tag_list=[]
        for tag in tag_html:
            tag_list.append(tag.text)
        df = pd.concat([df, pd.DataFrame([{'text': text, 'author':author, 'tags': tag_list}])], ignore_index=True)
    

In [22]:
df.count()

text      100
author    100
tags      100
dtype: int64

La web tiene 10 citas por página, así que podemos ver que ha guardado el contenido de toda la web en el dataframe. A continuación vamos a almacenar nuestro dataframe en un csv, para poderlo usar en otro momento.

In [23]:
df.to_csv("bs_quotes.csv")

Ahora repetiremos el proceso pero con Selenium.

In [2]:
url="https://quotes.toscrape.com/"

In [3]:
opts = Options()
opts.add_argument('-headless=new')
browser = Firefox(options=opts)
browser.get(url)

In [6]:
quotes=browser.find_elements(By.CLASS_NAME, 'quote')

In [22]:
for quote in quotes:
    text=quote.find_element(By.CLASS_NAME, 'text').text
    author=quote.find_element(By.CLASS_NAME, 'author').text
    tag_html=quote.find_elements(By.CLASS_NAME, 'tags')
    tag_list=[]
    for tag in tag_html:
        tag_list.append(tag.text)
    print(text, author, tag_list)

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.” Albert Einstein ['Tags: change deep-thoughts thinking world']
“It is our choices, Harry, that show what we truly are, far more than our abilities.” J.K. Rowling ['Tags: abilities choices']
“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.” Albert Einstein ['Tags: inspirational life live miracle miracles']
“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.” Jane Austen ['Tags: aliteracy books classic humor']
“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.” Marilyn Monroe ['Tags: be-yourself inspirational']
“Try not to become a man of success. Rather become a man of value.” Albert Einstein ['Tags: adulthood success value']
“It is better to be hated for what you are than to be love

Tal como hemos hecho con Beautiful Soup ahora haremos que visite las 10 páginas que forman la web y meteremos el contenido en un dataframe

In [23]:
df = pd.DataFrame(columns=['text', 'author', 'tags'])

In [24]:
for i in range(1,11):
    url="https://quotes.toscrape.com/page/" + str(i) + "/"
    browser.get(url)
    quotes=browser.find_elements(By.CLASS_NAME, 'quote')
    for quote in quotes:
        text=quote.find_element(By.CLASS_NAME, 'text').text
        author=quote.find_element(By.CLASS_NAME, 'author').text
        tag_html=quote.find_elements(By.CLASS_NAME, 'tags')
        tag_list=[]
        for tag in tag_html:
            tag_list.append(tag.text)
        df = pd.concat([df, pd.DataFrame([{'text': text, 'author':author, 'tags': tag_list}])], ignore_index=True)
    

In [25]:
df.head()

,text,author,tags
0,“The world as we have created it is a process ...,Albert Einstein,[Tags: change deep-thoughts thinking world]
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,[Tags: abilities choices]
2,“There are only two ways to live your life. On...,Albert Einstein,[Tags: inspirational life live miracle miracles]
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,[Tags: aliteracy books classic humor]
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,[Tags: be-yourself inspirational]


Y guardaremos los datos generados en un csv

In [26]:
df.to_csv("sel_quotes.csv")

A continuación realizaremos un scrapeo de Wikipedia, en concreto de este enlace sobre el actor escocés David Tennant https://es.wikipedia.org/wiki/David_Tennant. Queremos guardar los datos de que premios y nominaciones ha recibido a lo largo de su carrera.

Empezaremos haciendo el proceso con Beautiful Soup

In [126]:
url="https://es.wikipedia.org/wiki/David_Tennant"

In [127]:
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")

Seleccionamos la tablas del artículo

In [128]:
tables=soup.find_all("table", class_="wikitable")

La tabla que buscamos es la quinta del artículo (indice 4)

In [129]:
awards_table=tables[4]

In [130]:
print(awards_table)

<table class="wikitable sortable">
<tbody><tr>
<th>Año
</th>
<th>Trabajo
</th>
<th>Premio
</th>
<th>Categoría
</th>
<th>Resultado
</th></tr>
<tr>
<td>1995</td>
<td><i>An Experienced Woman Gives Advice</i></td>
<td>Premio de Teatro Manchester Evening News<sup class="reference separada" id="cite_ref-48"><a href="#cite_note-48"><span class="corchete-llamada">[</span>48<span class="corchete-llamada">]</span></a></sup>​</td>
<td>Mejor Actor</td>
<td rowspan="" style="background-color:#fdd;color:black; text-align:center; vertical-align:middle;">Nominado
</td></tr>
<tr>
<td>1996</td>
<td><i><a href="/wiki/El_zoo_de_cristal" title="El zoo de cristal">The Glass Menagerie</a></i></td>
<td>Theatre Management Association<sup class="reference separada" id="cite_ref-49"><a href="#cite_note-49"><span class="corchete-llamada">[</span>49<span class="corchete-llamada">]</span></a></sup>​</td>
<td>Mejor Actor</td>
<td rowspan="" style="background-color:#fdd;color:black; text-align:center; vertical-align:

Ya hemos accedido a la tabla que queremos que tiene todo el html y sus respectivos hijos de donde podremos sacar la información

In [131]:
headers=awards_table.findAll("th")
columns=[]
for header in headers:
    columns.append(header.text)
print(columns)

['Año\n', 'Trabajo\n', 'Premio\n', 'Categoría\n', 'Resultado\n']


Quitamos el salto de línea al final del nombre de las columnas

In [132]:
for i,column in enumerate(columns):
    columns[i]=column[:-1]
print(columns)

['Año', 'Trabajo', 'Premio', 'Categoría', 'Resultado']


Creamos el dataframe con las columnas

In [133]:
df=pd.DataFrame(columns=columns)

Cogeremos las filas de la tabla para el contenido

In [134]:
rows=awards_table.findAll("tr")

Recorreremos las filas con sus respectivas celdas e iremos llenando el dataframe

In [135]:
rows[0]

<tr>
<th>Año
</th>
<th>Trabajo
</th>
<th>Premio
</th>
<th>Categoría
</th>
<th>Resultado
</th></tr>

Eliminaremos la primera fila, que contiene los encabezados

In [136]:
rows=rows[1:]

In [137]:
rows[0]

<tr>
<td>1995</td>
<td><i>An Experienced Woman Gives Advice</i></td>
<td>Premio de Teatro Manchester Evening News<sup class="reference separada" id="cite_ref-48"><a href="#cite_note-48"><span class="corchete-llamada">[</span>48<span class="corchete-llamada">]</span></a></sup>​</td>
<td>Mejor Actor</td>
<td rowspan="" style="background-color:#fdd;color:black; text-align:center; vertical-align:middle;">Nominado
</td></tr>

In [138]:
for row in rows:
    cells=row.findAll("td")
    year=cells[0].text
    work=cells[1].text
    award=cells[2].text[:-5] #Eliminamos el número de nota al final
    category=cells[3].text
    result=cells[4].text[:-1] #Eliminamos un salto de linea al final
    df=pd.concat([df, pd.DataFrame([{'Año': year, 'Trabajo' : work, 'Premio' : award, 'Categoría': category, 'Resultado' : result}])])
    

In [139]:
df.head()

,Año,Trabajo,Premio,Categoría,Resultado
0,1995,An Experienced Woman Gives Advice,Premio de Teatro Manchester Evening News,Mejor Actor,Nominado
0,1996,The Glass Menagerie,Theatre Management Association,Mejor Actor,Nominado
0,2000,The Comedy of Errors,Premio Ian Charleson,Best Classical Actor Under 30,Nominado
0,2003,Lobby Hero,Premio Olivier,Mejor Actor,Nominado
0,2005,Look Back in Anger,Critics' Awards for Theatre in Scotland,Best Male Performance,Ganador


Ya tenemos nuestro dataframe listo para ser almacenado en un csv

In [140]:
df.to_csv("bs_david_tennant.csv")